# **ANÁLISIS EXPLORATORIO DE DATOS**

Tras probar con varios modelos e hiperparámetros hemos concluido que el modelo de regresión lineal con hiperparámetros (fit_intercept= True) es el mejor. Tiene los errores rmse y mae más pequeños y es el más rápido. Vamos a entrenar el modelo con el archivo disp para finalmente predecir los valores de la variable de salida de comp.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore') # Para evitar los molestos avisos.


In [ ]:
# Descomprimimos los archivos
df_disp = pd.read_csv('disp_st6ns1.txt.bz2', compression="bz2", index_col=0)
df_comp = pd.read_csv('comp_st6ns1.txt.bz2', compression="bz2", index_col=0)
pd.set_option('display.max_columns', None)
df_comp['salida'] = None
df_comp.head(3650)

,apcp_sf1_1,apcp_sf2_1,apcp_sf3_1,apcp_sf4_1,apcp_sf5_1,dlwrf_s1_1,dlwrf_s2_1,dlwrf_s3_1,dlwrf_s4_1,dlwrf_s5_1,dswrf_s1_1,dswrf_s2_1,dswrf_s3_1,dswrf_s4_1,dswrf_s5_1,pres_ms1_1,pres_ms2_1,pres_ms3_1,pres_ms4_1,pres_ms5_1,pwat_ea1_1,pwat_ea2_1,pwat_ea3_1,pwat_ea4_1,pwat_ea5_1,spfh_2m1_1,spfh_2m2_1,spfh_2m3_1,spfh_2m4_1,spfh_2m5_1,tcdc_ea1_1,tcdc_ea2_1,tcdc_ea3_1,tcdc_ea4_1,tcdc_ea5_1,tcolc_e1_1,tcolc_e2_1,tcolc_e3_1,tcolc_e4_1,tcolc_e5_1,tmax_2m1_1,tmax_2m2_1,tmax_2m3_1,tmax_2m4_1,tmax_2m5_1,tmin_2m1_1,tmin_2m2_1,tmin_2m3_1,tmin_2m4_1,tmin_2m5_1,tmp_2m_1_1,tmp_2m_2_1,tmp_2m_3_1,tmp_2m_4_1,tmp_2m_5_1,tmp_sfc1_1,tmp_sfc2_1,tmp_sfc3_1,tmp_sfc4_1,tmp_sfc5_1,ulwrf_s1_1,ulwrf_s2_1,ulwrf_s3_1,ulwrf_s4_1,ulwrf_s5_1,ulwrf_t1_1,ulwrf_t2_1,ulwrf_t3_1,ulwrf_t4_1,ulwrf_t5_1,uswrf_s1_1,uswrf_s2_1,uswrf_s3_1,uswrf_s4_1,uswrf_s5_1,salida
V4381,0.000000,0.000000,0.000000,0.000000,0.000000,261.677721,260.085538,266.981640,290.442300,288.198866,0.0,10.000000,170.909091,469.090909,314.545455,100897.157670,100674.546165,100356.840909,99959.828835,100312.625000,12.532074,11.731115,12.357298,9.660853,10.655890,0.002468,0.002568,0.003115,0.003945,0.004326,0.011818,0.008182,0.014545,0.008182,0.000000,0.010336,0.007673,0.015555,0.008209,0.001336,281.266463,279.781200,287.694009,291.729925,291.755030,279.441673,279.048975,279.049155,287.887953,285.737038,279.439758,279.791043,287.696397,291.697798,285.750938,277.448223,279.017020,291.347043,293.756936,283.195601,338.810772,336.359000,357.210272,421.432717,407.708593,242.914279,238.327445,243.430893,265.033578,265.722910,0,4.000000,39.454545,99.363636,69.727273,None
V4382,0.000000,0.000000,0.000000,0.000000,0.000000,250.657744,231.172975,235.701976,256.459174,257.397292,0.0,19.090909,179.090909,476.363636,315.454545,101386.270597,101547.748580,101401.065341,101009.365057,100987.076705,6.345163,5.484385,6.156194,7.330083,7.836364,0.002937,0.002979,0.002551,0.002293,0.002507,0.000000,0.003636,0.007273,0.018182,0.022727,0.000764,0.005445,0.008036,0.016964,0.024418,282.334523,277.417977,282.996887,284.811931,284.832361,277.476773,276.252769,276.253806,283.100647,280.695884,277.476252,277.031109,283.001279,284.818506,280.706155,275.981476,276.772367,288.315577,288.789653,278.245736,340.235856,326.145541,345.760856,398.907898,383.254561,233.988552,241.998055,241.536482,234.666532,226.737294,0,4.000000,42.181818,100.818182,68.636364,None
V4383,0.000000,0.000000,0.000000,0.000000,0.000000,262.820357,249.579068,257.462440,280.814650,278.613109,0.0,10.000000,169.090909,480.909091,321.818182,100775.949574,100771.593040,100586.460227,100206.063920,100214.311080,8.906386,9.473962,10.063274,10.047274,9.870472,0.001818,0.002235,0.002113,0.002450,0.002501,0.004545,0.017273,0.010909,0.009091,0.020909,0.004955,0.016836,0.010827,0.009709,0.022091,277.853987,277.244002,285.542184,289.431072,289.469494,274.295216,273.645291,273.646928,285.717477,283.819092,274.321422,277.243084,285.545285,289.425881,283.826760,272.146312,276.060403,291.279666,292.280185,280.451116,322.429829,311.904067,343.689650,417.928189,400.278889,189.063528,225.864476,226.842826,255.776679,257.641826,0,3.272727,38.272727,101.090909,70.909091,None
V4384,0.000000,0.000000,0.102727,0.588182,0.672727,268.840246,264.601809,265.773438,310.059895,307.676752,0.0,20.000000,177.272727,345.454545,225.454545,99425.846591,99331.265625,99210.033381,99415.950994,99871.959517,10.901706,6.442958,12.324988,12.528385,10.829392,0.002954,0.003249,0.004204,0.004139,0.003514,0.056364,0.007273,0.063636,0.215455,0.117273,0.056982,0.006427,0.065264,0.215055,0.116945,282.993963,283.785708,288.347620,288.964727,288.996540,279.871429,282.430936,282.431366,287.281946,284.865232,282.934493,283.046894,288.341026,287.796603,284.863120,280.641460,281.662839,291.165527,288.568953,283.064356,342.151670,353.230241,368.851157,402.159651,391.558044,229.728516,243.084564,257.023654,236.805474,224.370216,0,4.000000,42.000000,67.909091,44.636364,None
V4385,0.000000,0.000000,0.000000,0.000000,0.0

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Separamos los datos en train y test
x = df_disp.loc[:,df_disp.columns != 'salida']
y = df_disp['salida']
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=3650, test_size=730, shuffle=False, random_state=6)

# Escalamos los datos
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time

# Ajustar el modelo de regresión lineal a los datos de entrenamiento
start = time.time()
modelo = LinearRegression(fit_intercept= True)
modelo.fit(x_train, y_train)
end = time.time()

# Realizar predicciones sobre los datos de prueba
y_pred = modelo.predict(x_test)

# Calcular el RMSE y MAE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

# Imprimir los resultados
print("RMSE:", rmse)
print("MAE:", mae)
print("Tiempo de ejecucion: ", end-start)


RMSE: 2990976.49666841
MAE: 2010353.8258214118
Tiempo de ejecucion:  0.02426290512084961


Hemos entrenado nuestro modelo con los datos de disp para que esté listo y pueda predecir los valores de salida del nuevo conjunto de datos. Es importante tener en cuenta los valores del RMSE y MAE, ya que nos indican qué tan precisas son las predicciones de nuestro modelo.

In [ ]:
import pandas as pd
import joblib
from sklearn.linear_model import LinearRegression

# Cargar conjunto de datos
np.set_printoptions(precision=None, suppress=True)

# Dividir en características (X) y variable objetivo (y)
X = df_disp.drop('salida', axis=1)
y = df_disp['salida']

# Utilizar el modelo para predecir la variable de salida para un nuevo conjunto de datos
new_X = df_comp.drop('salida', axis=1)
new_y_pred = modelo.predict(new_X)

df_comp['salida'] = new_y_pred


# Guardar el modelo entrenado en un archivo
joblib.dump(modelo, 'modelo_final.pkl')


['modelo_final.pkl']

In [ ]:
df_comp.head(3650)

,apcp_sf1_1,apcp_sf2_1,apcp_sf3_1,apcp_sf4_1,apcp_sf5_1,dlwrf_s1_1,dlwrf_s2_1,dlwrf_s3_1,dlwrf_s4_1,dlwrf_s5_1,dswrf_s1_1,dswrf_s2_1,dswrf_s3_1,dswrf_s4_1,dswrf_s5_1,pres_ms1_1,pres_ms2_1,pres_ms3_1,pres_ms4_1,pres_ms5_1,pwat_ea1_1,pwat_ea2_1,pwat_ea3_1,pwat_ea4_1,pwat_ea5_1,spfh_2m1_1,spfh_2m2_1,spfh_2m3_1,spfh_2m4_1,spfh_2m5_1,tcdc_ea1_1,tcdc_ea2_1,tcdc_ea3_1,tcdc_ea4_1,tcdc_ea5_1,tcolc_e1_1,tcolc_e2_1,tcolc_e3_1,tcolc_e4_1,tcolc_e5_1,tmax_2m1_1,tmax_2m2_1,tmax_2m3_1,tmax_2m4_1,tmax_2m5_1,tmin_2m1_1,tmin_2m2_1,tmin_2m3_1,tmin_2m4_1,tmin_2m5_1,tmp_2m_1_1,tmp_2m_2_1,tmp_2m_3_1,tmp_2m_4_1,tmp_2m_5_1,tmp_sfc1_1,tmp_sfc2_1,tmp_sfc3_1,tmp_sfc4_1,tmp_sfc5_1,ulwrf_s1_1,ulwrf_s2_1,ulwrf_s3_1,ulwrf_s4_1,ulwrf_s5_1,ulwrf_t1_1,ulwrf_t2_1,ulwrf_t3_1,ulwrf_t4_1,ulwrf_t5_1,uswrf_s1_1,uswrf_s2_1,uswrf_s3_1,uswrf_s4_1,uswrf_s5_1,salida
V4381,0.000000,0.000000,0.000000,0.000000,0.000000,261.677721,260.085538,266.981640,290.442300,288.198866,0.0,10.000000,170.909091,469.090909,314.545455,100897.157670,100674.546165,100356.840909,99959.828835,100312.625000,12.532074,11.731115,12.357298,9.660853,10.655890,0.002468,0.002568,0.003115,0.003945,0.004326,0.011818,0.008182,0.014545,0.008182,0.000000,0.010336,0.007673,0.015555,0.008209,0.001336,281.266463,279.781200,287.694009,291.729925,291.755030,279.441673,279.048975,279.049155,287.887953,285.737038,279.439758,279.791043,287.696397,291.697798,285.750938,277.448223,279.017020,291.347043,293.756936,283.195601,338.810772,336.359000,357.210272,421.432717,407.708593,242.914279,238.327445,243.430893,265.033578,265.722910,0,4.000000,39.454545,99.363636,69.727273,4.784885e+11
V4382,0.000000,0.000000,0.000000,0.000000,0.000000,250.657744,231.172975,235.701976,256.459174,257.397292,0.0,19.090909,179.090909,476.363636,315.454545,101386.270597,101547.748580,101401.065341,101009.365057,100987.076705,6.345163,5.484385,6.156194,7.330083,7.836364,0.002937,0.002979,0.002551,0.002293,0.002507,0.000000,0.003636,0.007273,0.018182,0.022727,0.000764,0.005445,0.008036,0.016964,0.024418,282.334523,277.417977,282.996887,284.811931,284.832361,277.476773,276.252769,276.253806,283.100647,280.695884,277.476252,277.031109,283.001279,284.818506,280.706155,275.981476,276.772367,288.315577,288.789653,278.245736,340.235856,326.145541,345.760856,398.907898,383.254561,233.988552,241.998055,241.536482,234.666532,226.737294,0,4.000000,42.181818,100.818182,68.636364,4.854915e+11
V4383,0.000000,0.000000,0.000000,0.000000,0.000000,262.820357,249.579068,257.462440,280.814650,278.613109,0.0,10.000000,169.090909,480.909091,321.818182,100775.949574,100771.593040,100586.460227,100206.063920,100214.311080,8.906386,9.473962,10.063274,10.047274,9.870472,0.001818,0.002235,0.002113,0.002450,0.002501,0.004545,0.017273,0.010909,0.009091,0.020909,0.004955,0.016836,0.010827,0.009709,0.022091,277.853987,277.244002,285.542184,289.431072,289.469494,274.295216,273.645291,273.646928,285.717477,283.819092,274.321422,277.243084,285.545285,289.425881,283.826760,272.146312,276.060403,291.279666,292.280185,280.451116,322.429829,311.904067,343.689650,417.928189,400.278889,189.063528,225.864476,226.842826,255.776679,257.641826,0,3.272727,38.272727,101.090909,70.909091,4.812604e+11
V4384,0.000000,0.000000,0.102727,0.588182,0.672727,268.840246,264.601809,265.773438,310.059895,307.676752,0.0,20.000000,177.272727,345.454545,225.454545,99425.846591,99331.265625,99210.033381,99415.950994,99871.959517,10.901706,6.442958,12.324988,12.528385,10.829392,0.002954,0.003249,0.004204,0.004139,0.003514,0.056364,0.007273,0.063636,0.215455,0.117273,0.056982,0.006427,0.065264,0.215055,0.116945,282.993963,283.785708,288.347620,288.964727,288.996540,279.871429,282.430936,282.431366,287.281946,284.865232,282.934493,283.046894,288.341026,287.796603,284.863120,280.641460,281.662839,291.165527,288.568953,283.064356,342.151670,353.230241,368.851157,402.159651,391.558044,229.728516,243.084564,257.023654,236.805474,224.370216,0,4.000000,42.000000,67.909091,44.636364,4.784261e+11
V4385,0.00000

Después de entrenar y evaluar nuestro modelo, hemos logrado que prediga la variable de salida del nuevo conjunto de datos. Es importante tener en cuenta los valores del RMSE y MAE, ya que esto nos indica que los resultados obtenidos no serán completamente precisos.

In [ ]:
df_comp.to_csv('predicciones.csv', index=False)